# Timeseries and Portfolio Index Data Preparation

Selected stocks for portfolio

1. IOC
2. TCS
3. HDFCBANK
4. KOTAKBANK
5. LT
6. ITC

### Import the Dependencies

In [1]:
import pandas as pd
import plotly.graph_objects as go
from tabulate import tabulate
import seaborn as sns
from datetime import datetime
import numpy as np

### Import Master dataset and filter required data

In [2]:
data_source = pd.read_pickle('Ten_year_data_consolidated.pkl')

In [3]:
IOC = data_source.loc(axis=1)[:,'IOC.NS']
IOC.columns = IOC.columns.droplevel(1)
IOC = IOC.reset_index()

In [4]:
TCS = data_source.loc(axis=1)[:,'TCS.NS']
TCS.columns = TCS.columns.droplevel(1)
TCS = TCS.reset_index()

In [5]:
HDFCBANK = data_source.loc(axis=1)[:,'HDFCBANK.NS']
HDFCBANK.columns = HDFCBANK.columns.droplevel(1)
HDFCBANK = HDFCBANK.reset_index()

In [6]:
KOTAKBANK = data_source.loc(axis=1)[:,'KOTAKBANK.NS']
KOTAKBANK.columns = KOTAKBANK.columns.droplevel(1)
KOTAKBANK = KOTAKBANK.reset_index()

In [7]:
LT = data_source.loc(axis=1)[:,'LT.NS']
LT.columns = LT.columns.droplevel(1)
LT = LT.reset_index()

In [8]:
ITC = data_source.loc(axis=1)[:,'ITC.NS']
ITC.columns = ITC.columns.droplevel(1)
ITC = ITC.reset_index()

Create a new DataFrame on the basis of the close prices of the individual stocks

In [9]:
data_list = ['IOC', 'ITC', 'TCS', 'HDFCBANK', 'KOTAKBANK', 'LT']
data_list_tickers = ['IOC.NS', 'ITC.NS', 'TCS.NS', 'HDFCBANK.NS', 'KOTAKBANK.NS', 'LT.NS']

In [10]:
Portfolio_data = pd.DataFrame()

In [11]:
Portfolio_data['Date'] = IOC['Date']
Portfolio_data['ITC'] = ITC['Close']
Portfolio_data['IOC'] = IOC['Close']
Portfolio_data['LT'] = LT['Close']
Portfolio_data['TCS'] = TCS['Close']
Portfolio_data['HDFCBANK'] = HDFCBANK['Close']
Portfolio_data['KOTAKBANK'] = KOTAKBANK['Close']

In [12]:
Portfolio_data.head()

,Date,ITC,IOC,LT,TCS,HDFCBANK,KOTAKBANK
0,2012-04-02,151.300003,65.099998,592.200012,597.125000,264.174988,278.000000
1,2012-04-03,151.399994,66.599998,605.377747,589.000000,265.100006,279.725006
2,2012-04-04,151.600006,65.712502,597.911133,589.224976,263.274994,275.450012
3,2012-04-09,149.366669,64.125000,577.044434,582.049988,260.875000,267.000000
4,2012-04-10,152.899994,62.762501,571.244446,578.099976,262.424988,273.524994


### Weightage of each stock

Now that our stocks have been selected we go ahead and derive the weightage each stock needs to have in our portfolio.

We have aimed to spread the risk equally among all stocks on the basis of their close price on the last day i.e. March 30th 2022.

1. Create a Df with the prices on March 30, 2022
2. Assume we are buying Rs. 10,000 worth of each stock
3. Find the number of stocks worth 10,100 for each company
4. Find the percentage of each number of units with respect to total number of stocks bought.

In [13]:
#extract last row i.e. latest price as per our data
Weights = Portfolio_data.tail(1)
#create new df with stock and corresponding latest prices
Weights = Weights.drop('Date', axis =1)
Weights = Weights.reset_index()
Weights = Weights.drop('index', axis = 1)
Weights = Weights.T
Weights = Weights.reset_index()
Weights = Weights.rename(columns={'index': 'Stocks', 0 : 'Prices'})

In [14]:
#considering we bought 10,000rs worth of each stock, we calculate number of units required
Weights['Units'] = 10000 / Weights['Prices'] 

In [15]:
#based on percentage of units we calculate weights
Weights['Weights'] = Weights['Units']/ sum(Weights['Units'])

In [16]:
#setting the company name as index to make it easier to call
Weights = Weights.set_index('Stocks')

In [17]:
Weights.at['ITC', 'Weights']

0.2744511984103582

In [18]:
Weights

,Prices,Units,Weights
Stocks,,,
ITC,249.199997,40.128411,0.274451
IOC,117.199997,85.324234,0.583560
LT,1773.599976,5.638250,0.038562
TCS,3731.550049,2.679852,0.018328
HDFCBANK,1476.949951,6.770710,0.046307
KOTAKBANK,1763.099976,5.671828,0.038791


### Creating the Index

We then multiply the individual stock price with the corresponding weights and sum them up to get the Portfolio's Index prices.

In [19]:
Portfolio_data['Weighted_price'] = (Portfolio_data['ITC']*Weights.at['ITC', 'Weights'])+(Portfolio_data['IOC']*Weights.at['IOC', 'Weights'])+(Portfolio_data['LT']*Weights.at['LT', 'Weights'])+(Portfolio_data['HDFCBANK']*Weights.at['HDFCBANK', 'Weights'])+(Portfolio_data['TCS']*Weights.at['TCS', 'Weights'])+(Portfolio_data['KOTAKBANK']*Weights.at['KOTAKBANK', 'Weights'])

In [20]:
Portfolio_data.head()

,Date,ITC,IOC,LT,TCS,HDFCBANK,KOTAKBANK,Weighted_price
0,2012-04-02,151.300003,65.099998,592.200012,597.125000,264.174988,278.000000,136.312065
1,2012-04-03,151.399994,66.599998,605.377747,589.000000,265.100006,279.725006,137.683837
2,2012-04-04,151.600006,65.712502,597.911133,589.224976,263.274994,275.450012,136.686676
3,2012-04-09,149.366669,64.125000,577.044434,582.049988,260.875000,267.000000,133.772242
4,2012-04-10,152.899994,62.762501,571.244446,578.099976,262.424988,273.524994,133.975702


### 10 year returns of Portfolio

We go ahead and calculate the total returns we would have recieved if we chose to invest in these stocks at this weightage 10 years ago, i.e. April 1st 2012.

In [29]:
First_price = Portfolio_data.iloc[0, Portfolio_data.columns.get_loc('Weighted_price')]

In [30]:
Last_price = Portfolio_data.iloc[2462, Portfolio_data.columns.get_loc('Weighted_price')]

In [31]:
Portfolio_returns = (Last_price-First_price)/First_price

In [32]:
Portfolio_returns

2.010440992414696

We perform to small excercise to check how many units would need to be purchased given a total portflio value:

In [33]:
print("Amount to be invested: ")
Amt_investment = int(input()) 

while Amt_investment < sum(Weights['Prices']):
    print("Please input number greater than", np.ceil(sum(Weights['Prices'])))
    Amt_investment = int(input())


ITC_amount = Amt_investment*Weights.at['ITC', 'Weights']
IOC_amount = Amt_investment*Weights.at['IOC', 'Weights']
LT_amount = Amt_investment*Weights.at['LT', 'Weights']
TCS_amount = Amt_investment*Weights.at['TCS', 'Weights']
HDFCBANK_amount = Amt_investment*Weights.at['HDFCBANK', 'Weights']
KOTAKBANK_amount = Amt_investment*Weights.at['KOTAKBANK', 'Weights']

ITC_no = np.floor(ITC_amount/Weights.at['ITC', 'Prices'])
IOC_no = np.floor(IOC_amount/Weights.at['IOC', 'Prices'])
LT_no = np.floor(LT_amount/Weights.at['LT', 'Prices'])
TCS_no = np.floor(TCS_amount/Weights.at['TCS', 'Prices'])
HDFCBANK_no = np.floor(HDFCBANK_amount/Weights.at['HDFCBANK', 'Prices'])
KOTAKBANK_no = np.floor(KOTAKBANK_amount/Weights.at['KOTAKBANK', 'Prices'])

table = [["ITC", ITC_no], ["IOC", IOC_no],["LT", LT_no],["TCS", TCS_no],["HDFCBANK", HDFCBANK_no],["KOTACKBANK", KOTAKBANK_no]]

print("Number of shares to be purchased per stock is:")
print(tabulate(table, headers=['Stock', 'Units'],tablefmt= 'fancy_grid',numalign="right"))


Amount to be invested: 
100000
Number of shares to be purchased per stock is:
╒════════════╤═════════╕
│ Stock      │   Units │
╞════════════╪═════════╡
│ ITC        │     110 │
├────────────┼─────────┤
│ IOC        │     497 │
├────────────┼─────────┤
│ LT         │       2 │
├────────────┼─────────┤
│ TCS        │       0 │
├────────────┼─────────┤
│ HDFCBANK   │       3 │
├────────────┼─────────┤
│ KOTACKBANK │       2 │
╘════════════╧═════════╛


### Export the weighted price as our univaroate time series data for further analysis

In [35]:
forecast_data = Portfolio_data[['Date','Weighted_price']]

In [36]:
forecast_data.to_pickle("Timeseries_data.pkl")

In [29]:
Weights = Weights.reset_index()

In [31]:
Weights.to_pickle('Weights.pkl')